In [10]:
from ISLP import load_data
from ISLP.models import ModelSpec as MS

import statsmodels.api as sm

In [75]:
Weekly = load_data('Weekly')
print(Weekly.shape)
Weekly.head()

(1089, 9)


,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [76]:
def fit_model(data):
    #print('data shape:', data.shape)
    y = data['Direction'] == 'Up'

    columns = ['Lag1', 'Lag2']
    design = MS(columns)
    X = design.fit_transform(data)
    #X.head()

    model = sm.GLM(y, X, family=sm.families.Binomial())
    results = model.fit()

    #results.summary()
    params = results.params

    r = model.predict(params, X)
    
    return results

def predict_result(rows, results):
    columns = ['Lag1', 'Lag2']
    design = MS(columns)
    X = design.fit_transform(rows)
    #print(X)

    return results.predict(X).values[0]

In [83]:
# Fit the model with all data except the first one
# Use the fitted model to predict the direction of the first observation
n = Weekly.shape[0]
correct = 0
for i in range(n):
    #fitted_model = fit_model(Weekly.iloc[1:Weekly.shape[0]])
    data = Weekly.drop(i, axis=0, inplace=False)
    fitted_model = fit_model(data)
    r = predict_result(Weekly.iloc[i:i+1], fitted_model)
    # If r > 0.5, the prediction is 'Up'
    if (r >= 0.5 and Weekly['Direction'].iloc[i] == 'Up') or (r < 0.5 and Weekly['Direction'].iloc[i] == 'Down'):
        correct += 1

print('Score:', correct/n)

Score: 0.5500459136822773
